<a href="https://colab.research.google.com/github/gpulipati/t5-chatbot/blob/main/notebooks/gemini_15_flash_web_chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain_community tiktoken langchainhub chromadb langchain
!pip install --upgrade --quiet  langchain-google-genai

In [5]:
import os
import getpass
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ["USER_AGENT"] = "GChat/1.0"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("Enter your LangSmith API key: ")
google_api_key = getpass.getpass("Enter your Google API key: ")

#import google.generativeai as genai
#from google.colab import userdata
#GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
#LANGCHAIN_API_KEY=userdata.get('LANGCHAIN_API_KEY')
#from getpass import getpass
#api_key = getpass()

Enter your LangSmith API key: ··········
Enter your Google API key: ··········


In [6]:
#from langchain_google_genai import GoogleGenerativeAI
#llm = GoogleGenerativeAI(model="models/gemini-1.5-flash", google_api_key=GOOGLE_API_KEY)
#print(
#    llm.invoke( "Tell me a joke about Bears?" )
#)

import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings

#### INDEXING ####

# Load Documents
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# Embed
vectorstore = Chroma.from_documents(documents=splits,
                                    embedding=GoogleGenerativeAIEmbeddings(model="models/text-embedding-004", google_api_key=google_api_key))

retriever = vectorstore.as_retriever()

#### RETRIEVAL and GENERATION ####

# Prompt
prompt = hub.pull("rlm/rag-prompt")

# LLM
llm = ChatGoogleGenerativeAI(model="models/gemini-1.5-flash", google_api_key=google_api_key, temperature=0)

# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain
rag_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)

print(
    rag_chain.invoke(
        "What is supported models by Task Decomposition?"
    )
)


Task decomposition is supported by models that can decompose complex tasks into smaller, simpler steps. This can be achieved through prompting techniques like Chain of Thought (CoT) or Tree of Thoughts (ToT). These methods allow models to break down tasks into manageable sub-tasks, making them easier to solve. 

